# Parallel Processing Example

In [1]:
import pyblp
import numpy as np

pyblp.options.digits = 3
pyblp.options.verbose = False

In this example, we'll use parallel processing to compute elasticities market-by-market for a simple Logit problem configured with some of the fake cereal data from [Nevo (2000)](../../references.rst#nevo-2000).

In [3]:
product_data = np.recfromcsv(pyblp.data.NEVO_PRODUCTS_LOCATION, encoding='utf-8')
formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)')
problem = pyblp.Problem(formulation, product_data)
results = problem.solve()
results

Problem Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:00     2         0             1              0.0                0.0         +4.23E+05       NA      

Linear Estimates (Robust SEs in Parentheses):
Beta:    prices   
-----  -----------
        -3.00E+01 
       (+1.01E+00)

In [6]:
pyblp.options.verbose = True
with pyblp.parallel(2):
    elasticities = results.compute_elasticities()

Starting a pool of 2 processes ...
Started the process pool after 0:00:00.
Computing elasticities with respect to prices ...
Finished after 0:00:01.

Terminating the pool of 2 processes ...
Terminated the process pool after 0:00:00.


Solving a Logit problem does not require market-by-market computation, so parallelization does not change its estimation procedure. Although elasticity computation does happen market-by-market, this problem is very small, so there are no gains from parallelization in this example.

If the problem were much larger, running ``Problem.solve`` and ``ProblemResults.compute_elasticities`` under the ``with`` statement could substantially speed up estimation and elasticity computation.